Generic imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

Data load

In [ ]:
recordsUrl = '../data/records_150_64000.csv'

dataset = pd.read_csv(recordsUrl)
dataset.tail()

# dataset.pop("BiggerPrime")

Split dataset

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

Visualize columns correlations

In [ ]:
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap

fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig, auto_add_to_figure=False)
fig.add_axes(ax)
cmap = ListedColormap(sns.color_palette("husl", 256).as_hex())

sc = ax.scatter(train_dataset['NumberN'], train_dataset['SmallerPrime'], train_dataset['BiggerPrime'], cmap=cmap, alpha=1)
ax.set_title("3D plot")
ax.set_xlabel('NumberN')
ax.set_ylabel('SmallerPrime')
ax.set_zlabel('BiggerPrime')

In [ ]:
sns.pairplot(train_dataset, diag_kind='kde')

Evaluate column statistics

In [ ]:
train_dataset.describe().transpose()

Split outputs from inputs

In [ ]:
train_features = train_dataset.copy()
_ = train_features.pop('SmallerPrime')
_ = train_features.pop('BiggerPrime')
test_features = test_dataset.copy()
_ = test_features.pop('SmallerPrime')
_ = test_features.pop('BiggerPrime')


train_labels = train_dataset.copy()
_ = train_features.pop('NumberN')
test_labels = test_dataset.copy()
_ = test_labels.pop('NumberN')

In [ ]:
# model = tf.keras.Sequential([
#     keras.layers.Dense(activation=tf.nn.sigmoid, input_shape=(train_features.shape[1],)),
# ])
model = keras.Sequential([
    keras.Input(shape=(32, 0)),
    keras.layers.Dense(units=9, activation='relu'),
    keras.layers.Dense(units=5, activation='relu'),
    keras.layers.Dense(units=2, activation='softmax')
])

model.build()
model.summary()
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
%%time
history = model.fit(
    train_features,
    train_labels,
    epochs=16,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 1])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)

plot_loss(history)

In [ ]:
test_results = {}

test_results['model'] = model.evaluate(test_features['NumberN'], test_labels, verbose=0)
x = tf.linspace(0.0, 1.0, 256)
primesPredictions = model.predict(x)
smallerPrimesPredictions = primesPredictions[:, 0]
biggerPrimesPredictions = primesPredictions[:, 1]

In [ ]:
def plot_a(x, primesPredictions):
  plt.scatter(train_features['NumberN'], train_labels, label='Data')
  plt.plot(x, primesPredictions, color='k', label='Predictions')
  plt.xlabel('NumberN')
  plt.ylabel('Prediction')
  plt.legend()

plot_a(x,smallerPrimesPredictions)